## Classification of text article

Links
* [Reuters newswire dataset](https://keras.io/datasets/)
* [Reuters text classification](https://www.bonaccorso.eu/2016/08/02/reuters-21578-text-classification-with-gensim-and-keras/)
* [Keras Reuters MLP example](https://github.com/keras-team/keras/blob/master/examples/reuters_mlp.py)